In [42]:
#| default_exp classes.Agent_PydanticAI_Expert

# AGENT: Pydantic_AI Expert
> Default description (change me)

In [43]:
#| export
import os
from dataclasses import dataclass

from typing import List

from supabase import Client as SupabaseClient
from openai import AsyncClient

from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel

import agent_mafia.utils as utils
import agent_mafia.client.MafiaError as amme

from agent_mafia.prompts.agent import pydantic_agent_system_prompt
from agent_mafia.routes.openai import generate_openai_embbedding

In [ ]:
#| hide
import nbdev
from openai import AsyncOpenAI
from IPython.display import Markdown, display


In [56]:
open_ai_key = os.environ["OPENAI_API_KEY"]
supabase_url = os.environ["SUPABASE_URL"]
supabase_service_key = os.environ["SUPABASE_SERVICE_KEY"]

model = OpenAIModel( "gpt-4o-mini-2024-07-18")

In [47]:
#| export

@dataclass
class PydanticAIDependencies:
    supabase: SupabaseClient
    openai_client: AsyncClient
    
pydantic_ai_expert = Agent(
    model=model,
    system_prompt=pydantic_agent_system_prompt,
    deps_type=PydanticAIDependencies,
    retries=2,
)

In [48]:
#| exporti

def format_supabase_chunks(data: List[dict]) -> List[str]:
    return [ f"# {doc['title']}\n\n{doc['content']}" for doc in data]

def format_supabase_page(data: List[dict]) -> List[str]:
    page_title = data[0]["title"].split(" - ")[0]

    formatted_content = [f"# {page_title}\n"]

    for chunk in data:
        formatted_content.append(chunk["content"])

    return "\n\n".join(formatted_content)

@pydantic_ai_expert.tool
async def retrieve_llm(ctx: RunContext[PydanticAIDependencies], user_query: str) -> str:
    """
    Retrieve relevant documentation chunks based on the query with RAG.

    Args:
        ctx: The context including the Supabase client and OpenAI client
        user_query: The user's question or query

    Returns:
        A formatted string containing the top 5 most relevant documentation chunks
    """

    try:
        query_embedding = await generate_openai_embbedding(
            user_query, ctx.deps.openai_client
        )

        # Query Supabase for relevant documents
        result = ctx.deps.supabase.rpc(
            "match_site_pages",
            {
                "query_embedding": query_embedding,
                "match_count": 5,
                "filter": {"source": "pydantic_ai_docs"},
            },
        ).execute()

        data = result.data

        if not data:
            return "No relevant documentation found."

        formatted_chunks = format_supabase_chunks(data=data)

        return "\n\n---\n\n".join(formatted_chunks)

    except Exception as e:
        message = utils.generate_error_message(e)
        print(message)
        return message
    
@pydantic_ai_expert.tool
async def list_documentation_pages(
    ctx: RunContext[PydanticAIDependencies],
) -> List[str]:
    """
    Retrieve a list of all available Pydantic AI documentation pages.

    Returns:
        List[str]: List of unique URLs for all documentation pages
    """

    try:
        result = (
            ctx.deps.supabase.from_("site_pages")
            .select("url")
            .eq("metadata->>source", "pydantic_ai_docs")
            .execute()
        )

        if not result.data:
            return []

        urls = sorted(set(doc["url"] for doc in result.data))
        return urls

    except amme.MafiaError as e:
        print(e)

        return []

@pydantic_ai_expert.tool
async def get_page_content(ctx: RunContext[PydanticAIDependencies], url: str) -> str:
    """
    Retrieve the full content of a specific documentation page by combining all its chunks.

    Args:
        ctx: The context including the Supabase client
        url: The URL of the page to retrieve

    Returns:
        str: The complete page content with all chunks combined in order
    """

    try:
        result = (
            ctx.deps.supabase.from_("site_pages")
            .select("title, content, chunk_number")
            .eq("url", url)
            .eq("metadata->>source", "pydantic_ai_docs")
            .order("chunk_number")
            .execute()
        )

        data = result.data

        if not data:
            return f"No content found for URL: {url}"

        return format_supabase_page(data)

    except amme.MafiaError as e:
        print(e)

        return e

In [68]:
#| eval : False
openai_client = AsyncOpenAI(api_key=open_ai_key)

supabase = SupabaseClient(
    supabase_url, 
    supabase_service_key
)

dependencies = PydanticAIDependencies(
        supabase=supabase, openai_client=openai_client
    )


res = await pydantic_ai_expert.run(
        user_prompt="how do i implement an pydantic ai agent?",
        deps=dependencies,
)

display(Markdown(res.data))

To implement a Pydantic AI agent, you can follow this basic structure provided in the documentation:

1. **Import Required Modules**: Start by importing the necessary modules from Pydantic.

   ```python
   from pydantic import BaseModel
   from pydantic_ai import Agent
   from pydantic_ai.models.openai import OpenAIModel
   from pydantic_ai.providers.openai import OpenAIProvider
   ```

2. **Set Up Your Model**: Define the model that your agent will use. You can specify the model name and the provider’s base URL.

   ```python
   ollama_model = OpenAIModel(
       model_name='your-model-name', 
       provider=OpenAIProvider(base_url='http://your-provider-url/v1')
   )
   ```

3. **Define Input Data Structure**: Use a Pydantic model to define the structure for the input data that the agent will handle.

   ```python
   class CityLocation(BaseModel):
       city: str
       country: str
   ```

4. **Create the Agent**: Instantiate an `Agent` with the model and specify the result type.

   ```python
   agent = Agent(model=ollama_model, result_type=CityLocation)
   ```

5. **Run the Agent**: Use the `run_sync` method to execute a query through the agent.

   ```python
   result = agent.run_sync('Where were the Olympics held in 2012?')
   print(result.data)
   ```

6. **Check Usage**: Optionally, you can display the usage statistics.

   ```python
   print(result.usage())
   ```

### Example Complete Code:
```python
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

# Define the model
ollama_model = OpenAIModel(
    model_name='your-model-name', 
    provider=OpenAIProvider(base_url='http://your-provider-url/v1')
)

# Define the result type
class CityLocation(BaseModel):
    city: str
    country: str

# Create agent
agent = Agent(model=ollama_model, result_type=CityLocation)

# Run the agent
result = agent.run_sync('Where were the Olympics held in 2012?')
print(result.data)
print(result.usage())
```

Make sure to replace `'your-model-name'` and `'http://your-provider-url/v1'` with the actual model name and your provider's URL.

Feel free to modify the input queries and data structures according to your specific use case!

In [67]:
#| hide
nbdev.nbdev_export('./Agent_PydanticAI_Expert.ipynynb')